In [ ]:
import netket as nk
import matplotlib.pyplot as plt
import numpy as np
import jax
import qutip as qt
from jax import numpy as jnp
import netket.experimental as nkx
import optax as opt
import flax.linen as nn

In [ ]:
L = 6
gamma_0 = 1.0
omega = 1.5
# set the spacing multiplier
d = 2.0
#calculate the wavevector
k= 2*np.pi * 0.08 *d

g = nk.graph.Chain(length=L, pbc=False)
#define the basis
h_space = nk.hilbert.Qubit(N=L)
hamiltonian = nk.operator.LocalOperator(h_space, dtype=np.complex64)


num_op = nk.operator.LocalOperator(h_space, dtype=np.complex64)
emission_rate = nk.operator.LocalOperator(h_space, dtype=np.complex64)
j_ops = []


for i in range(L):
    hamiltonian += omega * nk.operator.spin.sigmax(h_space, i)
        
    for j in range(L):
        if i != j:
            # coefficients calculated from the Green's tensor
            z_ij = abs(i - j)
            G = -3/4 * gamma_0 * np.exp(1j* k * z_ij)/ (k*z_ij)**3 * (k**2 * z_ij**2 + 1j * k * z_ij - 1)
            J = -1 * np.real(G)
            Gamma = 2* G.imag
                
            hamiltonian += J/2 * (
                    nk.operator.spin.sigmap(h_space, i) * nk.operator.spin.sigmam(h_space, j) +
                    nk.operator.spin.sigmam(h_space, i) * nk.operator.spin.sigmap(h_space, j)
                )
            j_ops.append(Gamma/2 *(nk.operator.spin.sigmam(h_space, i) + nk.operator.spin.sigmam(h_space, j)))
                
            # add the correlation operator
            emission_rate += Gamma * (nk.operator.spin.sigmap(h_space, i) * nk.operator.spin.sigmam(h_space, j))
                
    # append the observable operators        
    num_op += (nk.operator.spin.sigmap(h_space, i) * nk.operator.spin.sigmam(h_space, i))/L


# create the Liouvillian superoperator
lind = nk.operator.LocalLiouvillian(hamiltonian, j_ops, dtype=np.complex64)

# create the ansatz model
rbm = nk.models.NDM(alpha=1, beta=4, use_visible_bias=True, use_hidden_bias=True, use_ancilla_bias=True, kernel_init=nn.initializers.normal(stddev=0.001),
                    bias_init=nn.initializers.normal(stddev=0.001), visible_bias_init=nn.initializers.normal(stddev=0.001))

# define the sampler
sampler = nk.sampler.MetropolisLocal(lind.hilbert, n_chains=16)
sampler_diag = nk.sampler.MetropolisLocal(h_space, n_chains=16)


# create the variational state
v_state = nk.vqs.MCMixedState(sampler, rbm, n_samples=4000, n_discard_per_chain=20,
                              n_samples_diag=2000, sampler_diag=sampler_diag,n_discard_per_chain_diag = 10, seed=789)
# Initialize the parameters of the variational state
v_state.init_parameters(jax.nn.initializers.normal(stddev=0.001), seed=987)



In [ ]:
corr_matrix = emission_rate.to_dense()

In [ ]:
# calcualte steady state density matrix with qutip
rho_ss = qt.steadystate(hamiltonian.to_qobj(), [j_op.to_qobj() for j_op in j_ops])
corr_matrix = emission_rate.to_dense()
num_matrix = num_op.to_dense()
rho_ss_matrix = rho_ss.full()
emission = np.real(np.trace(corr_matrix @ rho_ss_matrix))
print("The exact correalted emission is:", emission)

In [ ]:
# calculate observales with netket exact method
rho_nk = nk.exact.steady_state(lind, method='iterative')
num_op_matrix = num_op.to_sparse()
sx = np.trace(rho_nk @ num_op_matrix)
print(sx)
print(np.real(np.round(sx, 5)))
print(np.real(np.trace(rho_nk @ corr_matrix)))

In [ ]:
# defining the optimizer
optimizer = nk.optimizer.Sgd(0.1)

sr = nk.optimizer.SR(diag_shift=1e-2,solver=nk.optimizer.solver.svd, holomorphic=False)

# create the simulation object minimizing the Liouvillian
ss = nk.SteadyState(lind, variational_state=v_state, optimizer=optimizer, preconditioner=sr)

# save the data whicle running
logger= nk.logging.RuntimeLog()

# start the simulation
ss.run(out=logger, n_iter=1000, obs={"num": num_op, "emission": emission_rate})

In [ ]:
import numpy as np
from scipy.linalg import sqrtm

def fidelity(rho: np.array, sigma: np.array) -> float:
    """
    Calculate the fidelity between two density matrices.

    Parameters:
    rho (np.ndarray): First density matrix
    sigma (np.ndarray): Second density matrix

    Returns:
    float: Fidelity between rho and sigma
    """
    # Ensure rho and sigma are numpy arrays
    rho = np.array(rho)
    sigma = np.array(sigma)
    
    # Calculate the square root of rho
    sqrt_rho = sqrtm(rho)
    
    # Calculate the product sqrt_rho * sigma * sqrt_rho
    product = sqrt_rho @ sigma @ sqrt_rho
    
    # Calculate the square root of the product
    sqrt_product = sqrtm(product)
    
    # Calculate the trace of the square root of the product
    trace = np.trace(sqrt_product)
    
    # Calculate the fidelity
    fidelity_value = np.real(trace) ** 2
    
    return fidelity_value


In [ ]:
sig = v_state.to_matrix()
rho = np.array(rho_ss)
print("The fidelity is:", fidelity(rho, sig))

In [ ]:
# calculate fidelity with qutip matrix and variational state
sigma = v_state.to_qobj()
print(" The fidelity calculated with qutip is:", np.round(qt.fidelity(rho_ss, sigma),5))

In [ ]:
# convert the simulation data from the logger

saved_params = logger.data

x = saved_params["LdagL"]["iters"]
num = saved_params["num"]["Mean"]
corr = saved_params["emission"]["Mean"]

LdagL = saved_params["LdagL"]["Mean"]

In [ ]:
corr = np.round(corr.mean(), 6)/L / gamma_0
print("The correalted emission is;", corr)

In [ ]:
num_calc = num[-50:].mean().real
print("\n", "Atomic population:", np.round(num_calc,5))

In [ ]:
# plot the results

plt.plot(x, LdagL)
plt.title("LdagL")
plt.xlabel('Iteration')
plt.ylabel('<LdagL>')
plt.show()

plt.plot(x, num)
plt.axhline(y=sx, color='r', linestyle='--')
plt.title("Number Operator")
plt.xlabel('Iteration')
plt.ylabel('<Number Operator>')
plt.show()
